# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
# from mlagents_envs.environment import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [ ]:
env = UnityEnvironment(file_name="./Banana_Linux/Banana.x86_64")
# env = UnityEnvironment(file_name="./Banana_Linux_NoVis/Banana.x86_64")


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)


### 3. D3QN with Replay Buffer


In [ ]:
import numpy as np
import random
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim



class QNetwork(nn.Module):

    def __init__(self, state_size, action_size):
        super(QNetwork, self).__init__()
        torch.manual_seed(0)

        self.value = nn.Sequential(
            nn.Linear(state_size, 128),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

        self.advantage = nn.Sequential(
            nn.Linear(state_size, 128),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(128, action_size)
        )


    def forward(self, state):
        values = self.value(state)
        advantages = self.advantage(state)
        q_values = values + (advantages - advantages.mean())

        return q_values


class HyperParameters():

    def __init__(self):
        self.buffer_size = 10000
        self.batch_size = 64
        self.gamma = 0.99
        self.tau_soft = 0.001
        self.tau_hard = 100
        self.use_soft_update = True
        self.learning_rate = 0.0005 # 0.025 
        self.learning_step_count = 4

        self.eps_min = 0.03
        self.eps_deacy = 0.99995

        self.memory_epsilon = 0.01
        self.memory_alpha = 0.6
        self.memory_beta = 0.4

class Agent():

    def __init__(self, state_size, action_size, hp = HyperParameters()):

        print("Hyper Parameters: ")
        attrs = vars(hp)
        print('\n'.join("%s: %s " % item for item in attrs.items()))

        self.state_size = state_size
        self.action_size = action_size
        self.hp = hp
        random.seed(0)

        # Q-Networks
        self.qnetwork_local = QNetwork(state_size, action_size)
        self.qnetwork_target = QNetwork(state_size, action_size)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=self.hp.learning_rate)

        self.epsilon = 1.0

        self.criterion = torch.nn.CrossEntropyLoss()

        # Replay memory
        self.memory = Memory(hp.buffer_size, hp.memory_epsilon, hp.memory_alpha, hp.memory_beta)
        self.step_count = 0
    
    def step(self, state, action, reward, next_state, done):

        value = self.qnetwork_local(torch.Tensor(state))
        target = self.qnetwork_target(torch.Tensor(next_state))
        error = reward  + self.hp.gamma * target.max() - value[action]

        # Save error and experience in replay buffer
        self.memory.add(float(error), (state, action, reward, next_state, done))

        self.step_count += 1
        if self.step_count % self.hp.learning_step_count == 0:
            if self.memory.tree.write >= self.hp.batch_size:
                experience, tree_ids, weights = self.memory.sample(self.hp.batch_size)
                self.learn(experience, tree_ids, weights)

        self.update_target(self.hp.use_soft_update)

    def act(self, state):

        state = torch.from_numpy(state).float().unsqueeze(0)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        self.epsilon *= self.hp.eps_deacy
        self.epsilon = max(self.epsilon, self.hp.eps_min) 

        if random.random() > self.epsilon:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences, tree_ids, weights):

        states, actions, rewards, next_states, dones = experiences

        action = self.qnetwork_local.forward(states)

        action = torch.gather(action, 1, actions)
        target = self.qnetwork_target.forward(next_states)
        target = rewards + (self.hp.gamma * target.max(1)[0].unsqueeze(1) * (1- dones))

        self.optimizer.zero_grad()


        errors = target - action
        weights = torch.tensor(weights).unsqueeze(1)

        loss = (0.5 * errors.pow(2)) * weights
        loss = loss.mean()

        loss.backward()

        self.optimizer.step()

        # Update errors
        for idx, error in enumerate(errors.detach()):
            self.memory.tree.update(tree_ids[idx], error)


    def update_target(self, soft_update):
        if soft_update:
            for target_param, local_param in zip(self.qnetwork_target.parameters(), self.qnetwork_local.parameters()):
                target_param.data.copy_(self.hp.tau_soft*local_param.data + (1.0-self.hp.tau_soft)*target_param.data)
        elif self.step_count % self.hp.tau_hard == 0:
            for target_param, local_param in zip(self.qnetwork_target.parameters(), self.qnetwork_local.parameters()):
                target_param.data.copy_(local_param.data)



class SumTree:

    def __init__(self, capacity, epsilon, alpha):
        self.capacity = capacity
        self.tree = np.zeros( 2*capacity - 1 )
        self.data = np.zeros( capacity, dtype=object)
        self.write = 0

        self.epsilon = epsilon
        self.alpha = alpha

        self.is_full = False

    def _propagate(self, idx, change):
        parent = (idx - 1) // 2

        self.tree[parent] += change
        assert not math.isnan(self.tree[parent])

        if parent != 0:
            self._propagate(parent, change)

    def _retrieve(self, idx, s):
        left = 2 * idx + 1
        right = left + 1
        while left < len(self.tree):
            if s <= self.tree[left]:
                idx = left
            else:
                idx = right
                s = s-self.tree[left]

            left = 2 * idx + 1
            right = left + 1

        return idx


    def _compute_priority(self, error):
        return (abs(error) + self.epsilon) ** self.alpha


    def size(self):
        if self.is_full:
            return self.capacity
        return self.write

    def total(self):
        return self.tree[0]

    def add(self, error, data):

        idx = self.write + self.capacity - 1

        self.data[self.write] = data
        self.update(idx, error)

        self.write += 1
        if self.write >= self.capacity:
            self.write = 0
            self.is_full = True

    def update(self, idx, error):
        priority = self._compute_priority(error)
        change = priority - self.tree[idx]

        self.tree[idx] = priority

        while idx != 0:
            idx = (idx - 1) // 2
            self.tree[idx] += change

    def get(self, s):
        idx = self._retrieve(0, s)
        dataIdx = idx - self.capacity + 1
        return (idx, self.tree[idx], self.data[dataIdx])


class Memory:

    def __init__(self, capacity, epsilon, alpha, beta):
        self.tree = SumTree(capacity, epsilon, alpha)
        self.beta = beta


    def add(self, error, sample):
        self.tree.add(error, sample) 


    def sample(self, batch_size):

        min_priority = self.tree.tree.min() / self.tree.total()
        max_weight = (min_priority * self.tree.size()) ** (-self.beta)


        experiences = []
        tree_ids = []
        weights = []
        segment = self.tree.total() / batch_size

        for i in range(batch_size):
            a = segment * i
            b = segment * (i + 1)

            s = random.uniform(a, b)

            idx, priority, experience = self.tree.get(s)
            if experience != 0:
                experiences.append(experience)
                tree_ids.append(idx)
                norm_priority = priority / self.tree.total()
                weight = (norm_priority * self.tree.size()) ** (-self.beta)

                weights.append(weight / max_weight)

        if len(experiences) == 0:
            return

        states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float()
        actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long()
        rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float()
        next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float()
        dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float()


        return (states, actions, rewards, next_states, dones), tree_ids, weights


### 4. Run the agent in the Environment


In [ ]:
agent = Agent(state_size, action_size)


def plot_graph(scores):
    # plot the scores
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(len(scores)), scores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.savefig("score.png")
    plt.show()


agent = Agent(state_size, action_size)

def train(epoch):
    score_avg = 0
    scores = []
    scores_window = deque(maxlen=100)
    best_score = 0
    for i in range(epoch):
        env_info = env.reset(train_mode=True)[brain_name] # reset the environment

        state = env_info.vector_observations[0]            # get the current state
        score = 0                                          # initialize the score
        while True:
            action = agent.act(state)                      # select an action
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished

            agent.step(state, action, reward, next_state, done)
            score += reward                                # update the score
            state = next_state                             # roll over the state to next time step
            if done:                                       # exit loop if episode finished
                break
        scores_window.append(score)
        scores.append(score)
        print("\r {} Eps: {:.2f} Score: {}".format(i, agent.epsilon, score), end='')
        score_avg = np.mean(scores_window)
        if i > 100:
            score_avg /= 100
            print("\n {} Eps: {:.2f} Avg Score: {}\n".format(i, agent.epsilon, score_avg), end='')
            if score_avg >= 13 and score_avg > best_score:
                best_score = score_avg
                print("\nProblem solved : ", score_avg)
                plot_graph(scores)
                torch.save(agent.qnetwork_local.state_dict(), 'local.pth')
                torch.save(agent.qnetwork_target.state_dict(), 'target.pth')
                
            score_avg = 0


### 5 Watch a trained agent


In [ ]:

agent.qnetwork_local.load_state_dict(torch.load('local.pth'))
agent.qnetwork_target.load_state_dict(torch.load('target.pth'))


for i in range(10):
    env_info = env.reset(train_mode=True)[brain_name] # reset the environment

    state = env_info.vector_observations[0]            # get the current state
    score = 0                                          # initialize the score
    while True:
        action = agent.act(state)                      # select an action
        env_info = env.step(action)[brain_name]        # send the action to the environment
        next_state = env_info.vector_observations[0]   # get the next state
        reward = env_info.rewards[0]                   # get the reward
        done = env_info.local_done[0]                  # see if episode has finished

        agent.step(state, action, reward, next_state, done)
        score += reward                                # update the score
        scores.append(score)
        state = next_state                             # roll over the state to next time step
        if done:                                       # exit loop if episode finished
            break
        print("\r {} Eps: {:.2f} Score: {}".format(i, agent.epsilon, score), end='')
    score_avg += score
    if i % 100 == 0:
        score_avg /= 100
        print("\n {} Eps: {:.2f} Avg Score: {}\n".format(i, agent.epsilon, score_avg), end='')    
        score_avg = 0


When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```